## 1. Configuración del Entorno y Conexión a la Base de Datos

Para poder interactuar con la base de datos PostgreSQL desde nuestro notebook, primero debemos establecer una conexión. El siguiente código se encarga de configurar y centralizar el acceso a la base de datos utilizando **SQLAlchemy**.


In [2]:
import os
import pandas as pd 
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import create_engine

load_dotenv()

DB_HOST=os.getenv("DB_HOST")
DB_PORT=os.getenv("DB_PORT")
DB_NAME=os.getenv("DB_NAME")
DB_USER=os.getenv("DB_USER")
DB_PASSWORD=os.getenv("DB_PASSWORD")

DATABASE_URL = f"postgresql+pg8000://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

try:
    engine = create_engine(DATABASE_URL, echo=False, client_encoding='utf8')

    print("Motor db SQLAlchemy creado exitosamente para Docker")
except Exception as e:
    print(f"Error al crear db: {e}")
    raise

base = declarative_base()
DBSession = sessionmaker(bind=engine)

def get_db_engine():
    return engine


def get_db_session():
    return DBSession


def get_db_connection():
    return engine.connect()

Motor db SQLAlchemy creado exitosamente para Docker


### 2. Conteo General de Registros

Como primer paso de la exploración, realizamos un conteo de registros para cada tabla. Esto nos da una perspectiva inicial del volumen de los datos y nos sirve como una verificación rápida para asegurar que todas las tablas se cargaron correctamente desde los archivos `.sql`.

In [2]:
tablas = [
    "usuarios", "categorias", "productos", "ordenes", "detalleordenes",
    "direccionesenvio", "carrito", "metodospago", "ordenesmetodospago",
    "reseñasproductos", "historialpagos"
]

print("📊 Conteo de registros por tabla:")
with engine.connect() as conn:
    for tabla in tablas:
        total = pd.read_sql(f"SELECT COUNT(*) AS total FROM {tabla}", conn).iloc[0]["total"]
        print(f"✔️ {tabla}: {total} registros")


📊 Conteo de registros por tabla:
✔️ usuarios: 1000 registros
✔️ categorias: 12 registros
✔️ productos: 36 registros
✔️ ordenes: 10000 registros
✔️ detalleordenes: 10000 registros
✔️ direccionesenvio: 1000 registros
✔️ carrito: 5000 registros
✔️ metodospago: 7 registros
✔️ ordenesmetodospago: 10000 registros
✔️ reseñasproductos: 10000 registros
✔️ historialpagos: 10000 registros


### 3. Análisis de Integridad: Nulos en Claves Primarias (PK)


A continuación, se define un proceso automatizado que itera sobre cada tabla principal y su respectiva clave primaria para verificar la existencia de nulos.

In [7]:
# Diccionario con claves primarias de cada tabla
pk_por_tabla = {
    "usuarios": "usuarioid",
    "categorias": "categoriaid",
    "productos": "productoid",
    "ordenes": "ordenid",
    "detalleordenes": "detalleid",
    "direccionesenvio": "direccionid",
    "carrito": "carritoid",
    "metodospago": "metodopagoid",
    "ordenesmetodospago": "ordenmetodoid",
    "reseñasproductos": "reseñaid",
    "historialpagos": "pagoid"
}

# Función para chequear nulos en la clave primaria
def check_pk_nulls(table, pk_column):
    query = f"""
    SELECT COUNT(*) AS null_count
    FROM {table}
    WHERE "{pk_column}" IS NULL;
    """
    return pd.read_sql(query, engine)

# Verificación para todas las tablas
for tabla, pk_col in pk_por_tabla.items():
    print(f"\n🔍 Nulos en PK de {tabla.upper()} ({pk_col}):")
    try:
        result = check_pk_nulls(tabla, pk_col)
        display(result)
    except Exception as e:
        print(f"⚠️ Error en tabla {tabla}: {e}")





🔍 Nulos en PK de USUARIOS (usuarioid):


,null_count
0,0



🔍 Nulos en PK de CATEGORIAS (categoriaid):


,null_count
0,0



🔍 Nulos en PK de PRODUCTOS (productoid):


...

,null_count
0,0



🔍 Nulos en PK de RESEÑASPRODUCTOS (reseñaid):


,null_count
0,0



🔍 Nulos en PK de HISTORIALPAGOS (pagoid):


,null_count
0,0


### 4. Análisis de Integridad: Duplicados en Claves Primarias (PK)

El siguiente script revisa cada tabla para encontrar valores de claves primarias que se repitan más de una vez. Si una tabla devuelve un resultado vacío, significa que su integridad de unicidad es correcta.

In [3]:
# Diccionario con claves primarias de cada tabla
pk_por_tabla = {
    "usuarios": "usuarioid",
    "categorias": "categoriaid",
    "productos": "productoid",
    "ordenes": "ordenid",
    "detalleordenes": "detalleid",
    "direccionesenvio": "direccionid",
    "carrito": "carritoid",
    "metodospago": "metodopagoid",
    "ordenesmetodospago": "ordenmetodoid",
    "reseñasproductos": "reseñaid",
    "historialpagos": "pagoid"
}

# Revisión de duplicados en la PK
for tabla, pk_col in pk_por_tabla.items():
    print(f"\n📌 Duplicados en PK de {tabla.upper()} ({pk_col}):")
    query = f"""
    SELECT "{pk_col}", COUNT(*) AS cantidad
    FROM {tabla}
    GROUP BY "{pk_col}"
    HAVING COUNT(*) > 1
    ORDER BY cantidad DESC;
    """
    try:
        df = pd.read_sql(query, engine)
        if df.empty:
            print("✅ Sin duplicados encontrados.")
        else:
            display(df)
    except Exception as e:
        print(f"⚠️ Error en tabla {tabla}: {e}")



📌 Duplicados en PK de USUARIOS (usuarioid):
✅ Sin duplicados encontrados.

📌 Duplicados en PK de CATEGORIAS (categoriaid):
✅ Sin duplicados encontrados.

📌 Duplicados en PK de PRODUCTOS (productoid):
✅ Sin duplicados encontrados.

📌 Duplicados en PK de ORDENES (ordenid):
✅ Sin duplicados encontrados.

📌 Duplicados en PK de DETALLEORDENES (detalleid):
✅ Sin duplicados encontrados.

📌 Duplicados en PK de DIRECCIONESENVIO (direccionid):
✅ Sin duplicados encontrados.

📌 Duplicados en PK de CARRITO (carritoid):
✅ Sin duplicados encontrados.

📌 Duplicados en PK de METODOSPAGO (metodopagoid):
✅ Sin duplicados encontrados.

📌 Duplicados en PK de ORDENESMETODOSPAGO (ordenmetodoid):
✅ Sin duplicados encontrados.

📌 Duplicados en PK de RESEÑASPRODUCTOS (reseñaid):
✅ Sin duplicados encontrados.

📌 Duplicados en PK de HISTORIALPAGOS (pagoid):
✅ Sin duplicados encontrados.


### 5. Análisis de Integridad Referencial: Claves Foráneas Huérfanas

Para detectar estos casos, utilizamos `LEFT JOIN`: unimos la tabla hija con la padre y filtramos los casos donde la clave de la tabla padre es `NULL`, lo que indica una referencia rota.

In [6]:
import psycopg2
from dotenv import load_dotenv
import os

load_dotenv()

conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)

def check_orphans(query, label):
    with conn.cursor() as cur:
        cur.execute(query)
        rows = cur.fetchall()
        if not rows:
            print(f"✅ Sin claves huérfanas en {label}.")
        else:
            print(f"⚠️ Claves huérfanas encontradas en {label}:")
            for row in rows:
                print(row)

# Ordenes sin usuario
check_orphans("""
    SELECT o.* FROM ordenes o
    LEFT JOIN usuarios u ON o.usuarioid = u.usuarioid
    WHERE u.usuarioid IS NULL;
""", "ordenes")

# DetalleOrdenes sin orden o producto
check_orphans("""
    SELECT d.* FROM detalleordenes d
    LEFT JOIN ordenes o ON d.ordenid = o.ordenid
    WHERE o.ordenid IS NULL;
""", "detalleordenes → ordenes")

check_orphans("""
    SELECT d.* FROM detalleordenes d
    LEFT JOIN productos p ON d.productoid = p.productoid
    WHERE p.productoid IS NULL;
""", "detalleordenes → productos")

# Carrito sin usuario o producto
check_orphans("""
    SELECT c.* FROM carrito c
    LEFT JOIN usuarios u ON c.usuarioid = u.usuarioid
    WHERE u.usuarioid IS NULL;
""", "carrito → usuarios")

check_orphans("""
    SELECT c.* FROM carrito c
    LEFT JOIN productos p ON c.productoid = p.productoid
    WHERE p.productoid IS NULL;
""", "carrito → productos")

# Direcciones de envío sin usuario
check_orphans("""
    SELECT d.* FROM direccionesenvio d
    LEFT JOIN usuarios u ON d.usuarioid = u.usuarioid
    WHERE u.usuarioid IS NULL;
""", "direccionesenvio → usuarios")

# OrdenesMetodosPago sin orden o metodo
check_orphans("""
    SELECT omp.* FROM ordenesmetodospago omp
    LEFT JOIN ordenes o ON omp.ordenid = o.ordenid
    WHERE o.ordenid IS NULL;
""", "ordenesmetodospago → ordenes")

check_orphans("""
    SELECT omp.* FROM ordenesmetodospago omp
    LEFT JOIN metodospago mp ON omp.metodopagoid = mp.metodopagoid
    WHERE mp.metodopagoid IS NULL;
""", "ordenesmetodospago → metodospago")

# Reseñas sin usuario o producto
check_orphans("""
    SELECT r.* FROM reseñasproductos r
    LEFT JOIN usuarios u ON r.usuarioid = u.usuarioid
    WHERE u.usuarioid IS NULL;
""", "reseñasproductos → usuarios")

check_orphans("""
    SELECT r.* FROM reseñasproductos r
    LEFT JOIN productos p ON r.productoid = p.productoid
    WHERE p.productoid IS NULL;
""", "reseñasproductos → productos")

# HistorialPagos sin orden o metodo
check_orphans("""
    SELECT h.* FROM historialpagos h
    LEFT JOIN ordenes o ON h.ordenid = o.ordenid
    WHERE o.ordenid IS NULL;
""", "historialpagos → ordenes")

check_orphans("""
    SELECT h.* FROM historialpagos h
    LEFT JOIN metodospago mp ON h.metodopagoid = mp.metodopagoid
    WHERE mp.metodopagoid IS NULL;
""", "historialpagos → metodospago")

# Cierre conexión
conn.close()


✅ Sin claves huérfanas en ordenes.
✅ Sin claves huérfanas en detalleordenes → ordenes.
✅ Sin claves huérfanas en detalleordenes → productos.
✅ Sin claves huérfanas en carrito → usuarios.
✅ Sin claves huérfanas en carrito → productos.
✅ Sin claves huérfanas en direccionesenvio → usuarios.
✅ Sin claves huérfanas en ordenesmetodospago → ordenes.
✅ Sin claves huérfanas en ordenesmetodospago → metodospago.
✅ Sin claves huérfanas en reseñasproductos → usuarios.
✅ Sin claves huérfanas en reseñasproductos → productos.
✅ Sin claves huérfanas en historialpagos → ordenes.
✅ Sin claves huérfanas en historialpagos → metodospago.
